<a href="https://colab.research.google.com/github/lawrencejesse/2023-2034-Lawrence-Ranch-NDVI/blob/main/Sentinel2_RasterExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Packaging and Download

### Subtask:
Zip the exported GeoTIFFs in the `/content/exports` directory and provide a download link for the zip file.

**Reasoning**:
Use the `zip` command to create a zip archive of the exported files and then provide a link to download the created zip file.

In [35]:
import os
from google.colab import files

def zip_and_download_exports():
    export_dir = '/content/exports'
    zip_filename = 'ndvi_exports.zip'
    zip_filepath = f'/content/{zip_filename}'

    if os.path.exists(export_dir) and os.listdir(export_dir):
        # Create a zip archive of the exported files
        !zip -r $zip_filepath $export_dir

        # Provide a download link for the zip file
        print(f"Your files are ready for download. Click the link below to download {zip_filename}")
        files.download(zip_filepath)
    else:
        print("No files found in the export directory to zip.")

# You can call this function after the export tasks are complete and the files are in the /content/exports directory.
# Note: Earth Engine exports to Drive are asynchronous. You'll need to wait for them to complete before zipping.
# A more robust solution would involve monitoring the tasks, but for simplicity, you can manually run this cell after exports are finished.

## Data Export

### Subtask:
Implement the export logic, including handling individual image export and optional mean NDVI export.

**Reasoning**:
Define a function `export_ndvi` that takes the processed collection, AOI, start date, end date, and the mean NDVI toggle status as input. Inside the function, check if the mean NDVI toggle is on. If so, compute the mean NDVI for the collection and export it. Otherwise, iterate through the collection and export each image individually, clipping to the AOI and using descriptive filenames.

In [36]:
import os
import ee.batch

def export_ndvi(processed_collection, aoi, start_date, end_date, export_mean=False):
    """
    Exports the processed NDVI images as GeoTIFFs.

    Args:
        processed_collection (ee.ImageCollection): The processed Sentinel-2 image collection with NDVI.
        aoi (ee.FeatureCollection or ee.Geometry): The area of interest.
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.
        export_mean (bool): Whether to export the mean NDVI for the date range.
    """
    export_dir = '/content/exports'
    if not os.path.exists(export_dir):
        os.makedirs(export_dir)

    if export_mean:
        print("Exporting mean NDVI...")
        mean_ndvi_image = processed_collection.mean().clip(aoi)
        filename = f'NDVI_Mean_{start_date.replace("-", "")}_{end_date.replace("-", "")}.tif'
        task = ee.batch.Export.image.toDrive(
            image=mean_ndvi_image,
            description=filename,
            folder='earth_engine_exports', # You can change this folder name in your Google Drive
            fileNamePrefix=filename.replace('.tif', ''),
            scale=10,
            region=aoi.geometry(),
            fileFormat='GeoTIFF',
            crs='EPSG:4326'
        )
        task.start()
        print(f"Mean NDVI export task started: {task.id}")
    else:
        print("Exporting individual NDVI images...")
        image_list = processed_collection.toList(processed_collection.size())

        for i in range(image_list.size().getInfo()):
            image = ee.Image(image_list.get(i)).clip(aoi)
            image_date = ee.Image(image_list.get(i)).date().format('YYYYMMDD').getInfo()
            filename = f'NDVI_{image_date}.tif' # AOINAME is not available, using date only
            task = ee.batch.Export.image.toDrive(
                image=image,
                description=filename,
                folder='earth_engine_exports', # You can change this folder name in your Google Drive
                fileNamePrefix=filename.replace('.tif', ''),
                scale=10,
                region=aoi.geometry(),
                fileFormat='GeoTIFF',
                crs='EPSG:4326'
            )
            task.start()
            print(f"Export task for {filename} started: {task.id}")

## Sentinel-2 Data Processing

### Subtask:
Define a function to build the Sentinel-2 collection, apply a cloud mask using the SCL band, and compute the NDVI.

**Reasoning**:
Define a function `process_sentinel2` that takes the AOI, start date, and end date as input. Inside the function, filter the Sentinel-2 collection by date and bounds, apply the cloud mask, and compute the NDVI.

In [54]:
def process_sentinel2(aoi, start_date, end_date):
    """
    Builds the Sentinel-2 collection, applies cloud masking, and computes NDVI.

    Args:
        aoi (ee.FeatureCollection or ee.Geometry): The area of interest.
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.

    Returns:
        ee.ImageCollection: The processed Sentinel-2 image collection with NDVI.
    """
    print(f"Processing Sentinel-2 for AOI, dates: {start_date} to {end_date}") # Debug print
    # Filter Sentinel-2 SR Harmonized collection by date and bounds
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(aoi)

    print(f"Initial collection size: {collection.size().getInfo()}") # Debug print

    # Function to mask clouds using the Sentinel-2 SCL band
    def mask_clouds(image):
        scl = image.select('SCL')
        # These are the pixel values for clouds, cloud shadows, and snow.
        cloud_mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
        return image.updateMask(cloud_mask)

    # Apply cloud mask and compute NDVI
    def compute_ndvi(image):
        # Scale reflectance bands
        scaled_image = image.select(['B4', 'B8']).multiply(0.0001)
        ndvi = scaled_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        return image.addBands(ndvi)

    processed_collection = collection.map(mask_clouds).map(compute_ndvi)

    print(f"Processed collection size (after masking and NDVI): {processed_collection.size().getInfo()}") # Debug print

    return processed_collection.select('NDVI')

# Task
Build a Google Colab notebook in Python that allows users to upload a polygon file (KML, GeoJSON, or zipped Shapefile), parse it as an AOI in WGS84, preview the AOI on a map, select a date range using widgets, download cloud-masked Sentinel-2 NDVI GeoTIFFs for the AOI and date range (clipped to AOI if possible), save them with descriptive filenames to `/content/exports`, zip the files, and provide a download link. Include an optional toggle to export a single mean NDVI for the date range. The notebook should use `earthengine-api`, `geemap`, and `ipywidgets`, authenticate with Earth Engine, use Sentinel-2 SR Harmonized (COPERNICUS/S2_SR_HARMONIZED) with SCL cloud masking, compute NDVI after scaling reflectance by 0.0001, and output GeoTIFFs at 10m scale in EPSG:4326.

## Setup and authentication

### Subtask:
Install necessary libraries (`earthengine-api`, `geemap`, `ipywidgets`), authenticate with Earth Engine, and initialize it.


**Reasoning**:
Install the necessary libraries using pip.



In [55]:
%pip install -U -q earthengine-api geemap ipywidgets

**Reasoning**:
Authenticate and initialize Earth Engine.



In [56]:
import ee

try:
    ee.Initialize()
    print('Google Earth Engine has been successfully initialized.')
except Exception as e:
    print(f'Failed to initialize Google Earth Engine: {e}')
    print('Please authenticate with Google Earth Engine by running ee.Authenticate() in a separate cell.')


Failed to initialize Google Earth Engine: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.
Please authenticate with Google Earth Engine by running ee.Authenticate() in a separate cell.


**Reasoning**:
Authenticate Earth Engine as instructed by the error message.



In [57]:
ee.Authenticate()

True

**Reasoning**:
Re-initialize Earth Engine after successful authentication.



In [58]:
try:
    ee.Initialize()
    print('Google Earth Engine has been successfully initialized after authentication.')
except Exception as e:
    print(f'Failed to initialize Google Earth Engine after authentication: {e}')

Failed to initialize Google Earth Engine after authentication: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.


**Reasoning**:
Re-initialize Earth Engine with a project ID as instructed by the error message.



In [59]:
try:
    ee.Initialize(project='jessemapping')
    print('Google Earth Engine has been successfully initialized with a project.')
except Exception as e:
    print(f'Failed to initialize Google Earth Engine with a project: {e}')

Google Earth Engine has been successfully initialized with a project.


## Aoi upload and visualization

### Subtask:
Create functionality to upload a polygon file (KML, GeoJSON, or zipped Shapefile), parse it as an Earth Engine feature collection in WGS84, and display it on an interactive map using `geemap` with a satellite basemap, ensuring it's centered and visible.


**Reasoning**:
Import necessary libraries and create a file upload widget.



In [90]:
import ipywidgets as widgets
import geemap
import os

# Global variable to store the AOI name
aoi_name_global = None

# Define the handle_upload function here
def handle_upload(change):
    print("handle_upload function triggered.") # Debug print
    uploaded_file = upload_widget.value[0]
    file_name = uploaded_file['name']
    file_content = uploaded_file['content']

    global aoi # Declare aoi as global
    global aoi_name_global # Declare aoi_name_global as global
    aoi = None # Reset aoi
    aoi_name_global = None # Reset aoi name
    print(f"Processing file: {file_name}") # Debug print

    try:
        # Store the AOI name (cleaned filename without extension)
        aoi_name_global = os.path.splitext(file_name)[0]
        # Replace spaces and special characters if needed for a valid filename
        aoi_name_global = aoi_name_global.replace(" ", "_").replace("-", "_") # Example cleaning

        if file_name.endswith('.kml'):
            # Save content to a temporary KML file
            with open(file_name, 'wb') as f:
                f.write(file_content)
            aoi = geemap.kml_to_ee(file_name)
            os.remove(file_name) # Clean up temporary file
        elif file_name.endswith('.geojson'):
            # Save content to a temporary GeoJSON file
            with open(file_name, 'wb') as f:
                f.write(file_content)
            aoi = geemap.geojson_to_ee(file_name)
            os.remove(file_name) # Clean up temporary file
        elif file_name.endswith('.zip'):
            # Save content to a temporary zip file
            with open(file_name, 'wb') as f:
                f.write(file_content)
            aoi = geemap.shp_to_ee(file_name)
            os.remove(file_name) # Clean up temporary file
        else:
            print("Unsupported file type.")
            aoi_name_global = None # Reset if unsupported
            return

        if aoi:
            # Ensure the AOI is in WGS84 (EPSG:4326) with a non-zero error margin
            aoi = aoi.geometry().transform('EPSG:4326', 1) # Added maxError=1
            print("AOI uploaded and processed successfully.") # Debug print
            if aoi_name_global:
                print(f"AOI Name: {aoi_name_global}")
            print("Run the next cell to display the map.")
        else:
            print("Failed to process the uploaded file: AOI is None.") # Debug print
            aoi_name_global = None # Reset if processing fails

    except Exception as e:
        print(f"An error occurred during file processing: {e}") # Debug print
        aoi = None # Ensure aoi is None if processing fails
        aoi_name_global = None # Reset if processing fails


upload_widget = widgets.FileUpload(
    accept='.kml,.geojson,.zip',
    multiple=False
)

display(upload_widget)

# Attach the observer to the widget
upload_widget.observe(handle_upload, names='value')

FileUpload(value=(), accept='.kml,.geojson,.zip', description='Upload')

**Reasoning**:
Define the function to handle file uploads, parse the file as an Earth Engine feature collection, and display it on a map.



**Reasoning**:
Add a new cell to display the map with the uploaded AOI after the file has been processed by the `handle_upload` function.

In [61]:
# Display the AOI on a map after upload
if 'aoi' in globals() and aoi is not None:
    m = geemap.Map(basemap='SATELLITE')
    m.addLayer(ee.FeatureCollection(aoi), {}, 'Uploaded AOI')
    m.centerObject(aoi)
    display(m)
else:
    print("Please upload an AOI using the widget above first.")

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

## Interactive widgets for date and export

### Subtask:
Add `ipywidgets` for selecting a start and end date and a button to trigger the data export process. Include a toggle for exporting the mean NDVI.


In [92]:
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import ee
import ee.batch
import os

# Global variables used by functions in cell 8e6c47aa
# available_dates_dict = {}
# processed_collection_global = None

# The get_available_dates function and the on_click binding for get_dates_button
# have been moved to cell 8e6c47aa for better organization.

# This cell can now be used for other imports or definitions if needed,
# or it can be left with just necessary imports and global variable declarations.
# The core logic for date selection widgets and button interaction is now in cell 8e6c47aa.

# Note: The process_sentinel2 function is assumed to be defined in another cell (e.g., cell 6e2d21a4).
# Ensure that cell is run before running cell 8e6c47aa.

**Reasoning**:
Create and display the date picker widgets, the mean NDVI checkbox, and the export button as specified in the instructions.



In [93]:
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import ee
import ee.batch
import os

# Global variables to store available dates (string) and their corresponding time_start (milliseconds)
# available_dates_dict is now defined and populated within the get_available_dates function in this cell.
# processed_collection_global is now defined and populated within the get_available_dates function in this cell.
# aoi_name_global is defined and populated in cell b61aff89

# Function to get available dates - Moved from cell 32222955
def get_available_dates(b):
    print("DEBUG: get_available_dates function started.")
    global available_dates_dict, processed_collection_global # Declare as global
    if 'aoi' not in globals() or aoi is None:
        print("Please upload an AOI first.")
        return

    start_date_str = start_date_widget.value.strftime('%Y-%m-%d')
    end_date_str = end_date_widget.value.strftime('%Y-%m-%d')

    print("Fetching available dates...")
    # Assuming process_sentinel2 function is defined elsewhere (e.g., cell 6e2d21a4)
    processed_collection_global = process_sentinel2(aoi, start_date_str, end_date_str)

    # Get the dates and time_start from the processed collection
    print("Attempting to get dates and time_start from processed collection...")
    try:
        # Get the full image information to extract both date and time_start
        image_info_list = processed_collection_global.toList(processed_collection_global.size()).getInfo()

        available_dates_dict = {}
        available_date_strings = []
        for image_info in image_info_list:
            time_start_millis = image_info['properties']['system:time_start']
            date_str = datetime.utcfromtimestamp(time_start_millis / 1000).strftime('%Y-%m-%d')
            available_dates_dict[date_str] = time_start_millis
            available_date_strings.append(date_str)

        # Sort the dates
        available_date_strings.sort()
        available_dates_dict = {date: available_dates_dict[date] for date in available_date_strings}


        print(f"Successfully retrieved {len(available_dates_dict)} dates with time_start.")
    except Exception as e:
        print(f"Error getting dates and time_start: {e}")
        available_dates_dict = {}

    print(f"Found {len(available_dates_dict)} available dates.")
    if available_dates_dict:
        # Call the function to display the date selection and export widgets (defined below in this cell)
        display_date_selection_widgets()
    else:
        print("No cloud-free images found for the selected date range and AOI.")


# Function to create and display widgets for date selection and export - Remains in this cell
def display_date_selection_widgets():
    global date_selector, export_button, output_folder_widget

    clear_output(wait=True) # Clear previous widgets

    # Re-display the initial widgets (already in the output)
    display(start_date_widget, end_date_widget, mean_ndvi_toggle, get_dates_button)

    # Use the keys (date strings) from available_dates_dict for the SelectMultiple options
    available_date_strings = list(available_dates_dict.keys())

    # Create a SelectMultiple widget for date selection
    date_selector = widgets.SelectMultiple(
        options=available_date_strings,
        description='Select Dates:',
        disabled=False,
        rows=min(10, len(available_date_strings)), # Limit rows for readability
        layout=widgets.Layout(width='50%')
    )

    # Widget for specifying output folder
    output_folder_widget = widgets.Text(
        value='earth_engine_exports',
        description='Output Folder (Drive):',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )

    # Create the export button
    export_button = widgets.Button(
        description='Start Export',
        disabled=False,
        button_style='success',
        tooltip='Click to start the export process for selected dates',
        icon='download'
    )

    # Attach the export function to the button click event
    export_button.on_click(on_export_button_clicked)

    # Display the new widgets
    print("Available Dates:")
    display(date_selector, output_folder_widget, export_button)


# Define the export function - Remains in this cell
def on_export_button_clicked(b):
    global processed_collection_global, available_dates_dict, aoi_name_global # Declare as global
    if 'aoi' not in globals() or aoi is None:
        print("Please upload an AOI first.")
        return

    selected_dates = date_selector.value
    export_mean = mean_ndvi_toggle.value
    output_folder = output_folder_widget.value

    # Use a default AOI name if none was captured
    aoi_name = aoi_name_global if aoi_name_global else "AOI"


    if not selected_dates and not export_mean:
        print("Please select at least one date to export or choose to export the mean NDVI.")
        return

    if export_mean:
        print("Exporting mean NDVI...")
        mean_ndvi_image = processed_collection_global.mean().clip(aoi)
        start_date_str = start_date_widget.value.strftime('%Y-%m-%d')
        end_date_str = end_date_widget.value.strftime('%Y-%m-%d')
        # Updated filename format for mean NDVI
        filename = f'{aoi_name}_NDVI_Mean_{start_date_str}_{end_date_str}.tif'
        task = ee.batch.Export.image.toDrive(
            image=mean_ndvi_image,
            description=filename,
            folder=output_folder,
            fileNamePrefix=filename.replace('.tif', ''),
            scale=10,
            region=aoi,
            fileFormat='GeoTIFF',
            crs='EPSG:4326'
        )
        task.start()
        print(f"Mean NDVI export task started: {task.id}")
    else:
        print("Exporting selected individual NDVI images...")
        # Get the system:time_start milliseconds for the selected dates
        selected_time_starts = [available_dates_dict[date_str] for date_str in selected_dates if date_str in available_dates_dict]

        # Filter the processed collection by matching system:time_start with the exact milliseconds
        selected_collection = processed_collection_global.filter(
            ee.Filter.inList('system:time_start', selected_time_starts)
        )


        # Add a check for the size of the selected collection
        selected_collection_size = selected_collection.size().getInfo()
        print(f"Selected collection size after filtering: {selected_collection_size}") # Debug print

        if selected_collection_size == 0:
            print("No images found for the selected dates after filtering. Please check your date selections.")
            return

        image_list = selected_collection.toList(selected_collection.size()) # Use the actual size

        for i in range(image_list.size().getInfo()):
            image = ee.Image(image_list.get(i)).clip(aoi)
            # Get the date from the image properties
            image_ee_date = ee.Image(image_list.get(i)).date()
            # Format the date as YYYY-MM-DD
            image_date_str = image_ee_date.format('YYYY-MM-dd').getInfo()
            # Updated filename format for individual images
            filename = f'{aoi_name}_NDVI_{image_date_str}.tif'
            task = ee.batch.Export.image.toDrive(
                image=image,
                description=filename,
                folder=output_folder,
                fileNamePrefix=filename.replace('.tif', ''),
                scale=10,
                region=aoi,
                fileFormat='GeoTIFF',
                crs='EPSG:4326'
            )
            task.start()
            print(f"Export task for {filename} started: {task.id}")


# Create and display the initial widgets
start_date_widget = widgets.DatePicker(
    description='Start Date:',
    disabled=False,
    value=datetime(2023, 1, 1)
)

end_date_widget = widgets.DatePicker(
    description='End Date:',
    disabled=False,
    value=datetime(2023, 12, 31)
)

mean_ndvi_toggle = widgets.Checkbox(
    value=False,
    description='Export Mean NDVI Only',
    disabled=False,
    indent=False
)

get_dates_button = widgets.Button(
    description='Get Available Dates',
    disabled=False,
    button_style='info',
    tooltip='Click to find available cloud-free dates in the selected range',
    icon='search'
)

display(start_date_widget, end_date_widget, mean_ndvi_toggle, get_dates_button)

# Attach the get_available_dates function to the button click event - Moved from cell 32222955
# The get_dates_button widget is defined in this cell.
get_dates_button.on_click(get_available_dates)

# The initial widgets are displayed when the cell is run.
# The date selection and export widgets will be displayed after clicking "Get Available Dates"
# by calling display_date_selection_widgets which is defined in this cell.

DatePicker(value=datetime.date(2021, 1, 1), description='Start Date:', step=1)

DatePicker(value=datetime.date(2021, 12, 31), description='End Date:', step=1)

Checkbox(value=False, description='Export Mean NDVI Only', indent=False)

Button(button_style='info', description='Get Available Dates', icon='search', style=ButtonStyle(), tooltip='Cl…

Available Dates:


SelectMultiple(description='Select Dates:', layout=Layout(width='50%'), options=('2021-01-01', '2021-01-06', '…

Text(value='earth_engine_exports', description='Output Folder (Drive):', layout=Layout(width='50%'))

Button(button_style='success', description='Start Export', icon='download', style=ButtonStyle(), tooltip='Clic…

Exporting selected individual NDVI images...
Selected collection size after filtering: 1
Export task for AOI_NDVI_2021-06-15.tif started: 5Y2E4R6PYYBF5ST3GFNGSMOB


# Task
Build a Google Colab notebook in Python that allows users to upload a polygon file (KML, GeoJSON, or zipped Shapefile), parse it as an AOI in WGS84 (EPSG:4326), preview the AOI on a simple map with a Google Satellite basemap, select a date range using widgets, filter for cloud-free Sentinel-2 images within the AOI and date range, display the dates of available images for user selection, download the selected cloud-masked Sentinel-2 NDVI GeoTIFFs clipped to the AOI (or filtered by bounds), save the GeoTIFFs with descriptive filenames to `/content/exports`, zip the exported files, and provide a download link. The notebook should use `earthengine-api`, `geemap`, and `ipywidgets`, authenticate with Earth Engine, use Sentinel-2 SR Harmonized (COPERNICUS/S2_SR_HARMONIZED) and mask clouds with SCL, compute NDVI ((B8-B4)/(B8+B4)) after scaling reflectance by 0.0001, use a scale of 10m and EPSG:4326 output. Optionally, include a toggle to export a single mean NDVI for the selected dates.